In [ ]:
# установка пакетов
!pip install --upgrade ollama
!pip install --upgrade llama-index
!pip install --upgrade langchain
!pip install --upgrade langchain-ollama

In [1]:
OLLAMA_URL = "http://127.0.0.1:11434"
# OLLAMA_URL = "http://172.17.0.1:11434"

# Обращение к серверу с LLM

## Requests

In [2]:
import requests

try: 
    response = requests.get(OLLAMA_URL)
    if response.status_code == 200:         print("Ollama сервер работает!")
    else:                                   print(f"Сервер ответил с кодом: {response.status_code}")
except requests.ConnectionError:            print("Не удалось подключиться к Ollama серверу. Убедитесь, что он запущен.")

# список моделей
reps = requests.get( OLLAMA_URL + "/api/tags"  )
print(reps.text)

Ollama сервер работает!
{"models":[{"name":"rscr/vikhr_nemo_12b:Q4_K_M","model":"rscr/vikhr_nemo_12b:Q4_K_M","modified_at":"2024-12-26T20:48:38.657378091+09:00","size":7477220858,"digest":"a8149e66bb9329ea549a30443f850ed69456a443d156d9ec7f5ceb7fe380a1e2","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"12.2B","quantization_level":"Q4_K_M"}},{"name":"qwen2.5:7b","model":"qwen2.5:7b","modified_at":"2024-12-26T20:48:37.576363399+09:00","size":4683087332,"digest":"845dbda0ea48ed749caafd9e6037047aa19acfcfd82e704d7ca97d631a0b697e","details":{"parent_model":"","format":"gguf","family":"qwen2","families":["qwen2"],"parameter_size":"7.6B","quantization_level":"Q4_K_M"}},{"name":"mannix/llama3.1-8b-abliterated:q6_k","model":"mannix/llama3.1-8b-abliterated:q6_k","modified_at":"2024-12-26T20:48:36.441347972+09:00","size":6596021404,"digest":"77f40eeac40ba75b7adccff7dd17fbf41f05a4cb72ea4daa2f1100b029972fa4","details":{"parent_model":"","format":"g

In [3]:
data = {
       "model": "llama3.2:1b",
       "prompt": "What is the answer to the Ultimate Question of Life, the Universe, and Everything?",
    #    "prompt": "What model are you?",
       "stream": False,
}

# Отправляем POST-запрос
response = requests.post(OLLAMA_URL + "/api/generate", json=data)

# Проверяем, что запрос прошел успешно
if response.status_code == 200: print("Ответ от сервера:", response.json()['response'])
else:                           print("Ошибка при запросе:", response.status_code)

Ответ от сервера: The question you're referring to is a famous one from Douglas Adams' science fiction series "The Hitchhiker's Guide to the Galaxy." In the book, a supercomputer named Deep Thought is asked to find the answer to the ultimate question of life, the universe, and everything. After thinking for 7.5 million years, Deep Thought finally reveals that the answer is... 

**42**

However, the characters soon realize that they don't actually know what the ultimate question is, so the answer isn't really helpful in the grand scheme of things.

Do you want to discuss this with me further or move on to something else?


## ollama

In [4]:
import ollama

mes = 'Please translate from latin: Non dies sine committere'

response = ollama.chat(model='llama3.2:1b',
                      messages=[{'role': 'user', 'content': mes}])

print(response['message']['content'])

The translation of "Non dies sine committere" from Latin to English is:

"No days without working"

Or, more literally:

"No idle days" or "No free days".


## langchain_community.chat_models -> ChatOllama

**langchain** — это модульный фреймворк для работы с множеством языковых моделей, векторных баз данных и других инструментов обработки текста.

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage

# адрес и порт сервера Ollama
ollama_base_url = "http://localhost:11434"

# Инициализация модели
llm = ChatOllama(model="llama3.2:1b", base_url=ollama_base_url)

# Функция для получения ответа от модели
def get_response(user_input):
   messages = [HumanMessage(content=user_input)]
   return llm.invoke(messages).content

# Пример использования
user_input = "What is LD50 of coffee?"
response = get_response(user_input)
print("Ответ:", response)



## PromptTemplate

In [6]:
from langchain_core.prompts import PromptTemplate

# создание простого промпта
# todo: ...

# структура промпта
template = """
System: {system_message}\n\n
Human: {user_message}\n\n
Context: {context}\n\n
AI: Please respond to the human's message.
"""

# Последний раздел (AI) можно использовать по необходимости, когда запрос может быть не вполне очевиден или требует дополнительных уточнений.

prompt = PromptTemplate(
    template=template,
    input_variables=["system_message", "user_message", "context"]           # названия переменных из текста template
)

formatted_prompt = prompt.format(
    system_message="You are a helpful AI assistant.",
    user_message="What's the capital of France?",
    context="France is a country in Western Europe."
)

In [7]:
formatted_prompt

"\nSystem: You are a helpful AI assistant.\n\n\nHuman: What's the capital of France?\n\n\nContext: France is a country in Western Europe.\n\n\nAI: Please respond to the human's message.\n"

In [8]:
data = {
       "model": "llama3.2:1b",
       "prompt": formatted_prompt,
    #    "prompt": "What model are you?",
       "stream": False,
}

In [7]:
response = requests.post(f"{OLLAMA_URL}/api/generate", json=data)

In [8]:
response.json()['response']

'The capital of France is Paris.'

# Ссылки

- https://python.langchain.com/docs/integrations/llms/ollama/